In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
#   try:
#     tf.config.set_logical_device_configuration(
#         gpus[0],
#         [tf.config.LogicalDeviceConfiguration(memory_limit=8000)])
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

In [3]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat

import preprocess
import segmentation
import CNN

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat

import preprocess
import segmentation
import CNN

In [ ]:
data = loadmat('data/raw_training_data.mat')
print(data.keys())

In [ ]:
dg_data_r = data['train_dg']
ecog_data_r = data['train_ecog']

dg_data = []
ecog_data = []
for i in range(3):
    dg_data.append(dg_data_r[i][0])
    ecog_data.append(ecog_data_r[i][0])

In [ ]:
# baseline = find_b(test)

In [ ]:
baseline = []
for i in range(3):
    baseline.append(np.load('data/correction_baseline_' + str(i + 1) + '.npy'))

In [ ]:
dg_data_bl = [dg_data[i] - baseline[i] for i in range(len(dg_data))]

In [ ]:
thresh_data = preprocess.thresholding(dg_data_bl)

In [ ]:
norm_fit = preprocess.normalization_fit(thresh_data)
norm_thres_data = preprocess.normalize(thresh_data, norm_fit)
norm_bl_data = preprocess.normalize(dg_data_bl, norm_fit)
norm_raw_data = preprocess.normalize(dg_data, norm_fit)
norm_bl = preprocess.normalize(baseline, norm_fit)

In [ ]:
# for j in range(1):
#     for i in range(2):
#         print("Subject %i, Finger %i" % (j + 1, i + 1))
#         plt.figure(figsize=(15, 10))
#         plt.plot(norm_bl[j][:, i][:10000])
#         plt.plot(norm_raw_data[j][:, i][:10000])
#         # plt.plot(thresh_data[j][:,i][:100000])
#         plt.plot(norm_thres_data[j][:, i][:10000])
#         plt.show()

In [ ]:
seg = segmentation.get_windoweds(ecog_data[0], 1000, 1, 0.04)

In [ ]:
seg = seg[:, :, np.newaxis, :]
seg.shape

In [ ]:
seg_nn = segmentation.get_windoweds_stack(seg, 1, 100, 25)

In [ ]:
seg_nn.shape

In [ ]:
norm_bl_data[0][0:300000:40]

In [ ]:
index = np.array([i for i in range(7500)])
seg_dg = segmentation.get_windoweds_stack(index,1,25,1)
print(seg_dg.shape)
dg_instances = norm_bl_data[0][0:300000:40][:,0]

In [ ]:
dg_train = dg_instances[seg_dg]
dg_train.shape

In [ ]:
seg_dg_nn = segmentation.get_windoweds_stack(seg_dg[:,-1], 1, 100, 25)

In [ ]:
seg_dg_nn.shape

In [ ]:
# dg_train = dg_instances[seg_dg_nn]

In [ ]:
data_full = loadmat('data/sub1_comp')
dg_data_test = data_full['train_dg'][500:180000]
ecog_data_test = data_full['train_data'][500:180000]

In [ ]:
ecog_data_test.shape

In [ ]:
ecog_data_test = ecog_data_test[ :, np.newaxis, :]
seg_test = segmentation.get_windoweds_stack(ecog_data_test, 1000, 1, 0.04)
seg_test_nn = segmentation.get_windoweds_stack(seg_test, 1, 100, 10)

In [ ]:
print(seg_test.shape)
seg_test_nn.shape

In [ ]:
dg_test_instances = dg_data_test[0:179500:40][:,0]

In [ ]:
index = np.array([i for i in range(4487)])
seg_test_dg = segmentation.get_windoweds_stack(index,1,25,1)
print(seg_test_dg.shape)
seg_test_dg_nn = segmentation.get_windoweds_stack(seg_test_dg[:,-1], 1, 100, 10)
dg_test = dg_test_instances[seg_test_dg]

In [ ]:
dg_test.shape

In [ ]:
model = CNN.model(62)

In [ ]:
dg_train.shape

In [ ]:
# # with tf.device('/cpu:0'):
history = model.fit(seg, dg_train, epochs=10, batch_size=64,  verbose=1, validation_data=(seg_test, dg_test))

In [ ]:
print(history.history.keys())
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
with tf.device('/cpu:0'):
    pred = model.predict(seg)
pred.shape

In [ ]:
pred

In [ ]:
seg_dg

In [ ]:
# plt.plot(seg_dg.flatten(),pred.flatten())
plt.plot(pred[0])
plt.plot(dg_instances)
plt.ylabel('Angle')
plt.xlabel('Time Steps')
# plt.xlim((100,200))
plt.show()

In [ ]:
from scipy.stats import pearsonr
pearsonr(pred[:,-1], dg_instances[:7476])[0]

In [ ]:
pred[:,0].shape